In [1]:
import pandas as pd

In [2]:
from curate_sad_tracks import generate_sad_tracks

In [3]:
phenotype="Major Depressive Disorder"
gpt_track_list = generate_sad_tracks(phenotype=phenotype, no_lookup=False)

In [4]:
import gwas_snp_selection_fdr_bonf_method

In [5]:
# path to 1000genomes:
# maybe do a dummy file instead?
# path_1KG_directory = "Z:\data\projects\Lovro_GWAS_Power\GWAS\1000genomes_as_csv"
path_1KG_directory = "1kg_dummy"
# path to coding regions:
# path_exon = "Z:\data\projects\Lovro_GWAS_Power\GWAS\genome_assembly\exon_regions_v2.csv"
path_exon = "genome_assembly/coding_regions_with_snp_dummy.csv"

In [13]:
## Summary Stats Input
# does not seem to work:
# file_path_summary_stats = "Z:\data\projects\Lovro_GWAS_Power\GWAS\gwas_1_and_2_summary_statistics_data\GCST90277450.tsv"
file_path_summary_stats = "sumstats_head_1_2.tsv"

# open the file - must be .tsv.bgz or .tsv format
file = gzip.open(file_path_summary_stats, "rt") if file_path_summary_stats.endswith(".tsv.bgz") else open(file_path_summary_stats, "r")

# create a dataframe for the summary stats
selected_columns = ['chromosome', 'base_pair_location', 'effect_allele', 'other_allele', 'p_value']
df_sumstats = pd.concat(pd.read_csv(file, sep="\t", usecols=selected_columns, chunksize=100000),ignore_index=True)
# df_sumstats.rename(columns={'chromosome':'chr', 'base_pair_location':'pos', 'effect_allele':'ref', 'other_allele':'alt', 'p_value':'neglog10_pval_EUR'}, inplace=True)
df_sumstats.rename(columns={'chromosome':'chr', 'base_pair_location':'pos', 'effect_allele':'ref', 'other_allele':'alt', 'p_value':'p_value'}, inplace=True)

In [14]:
df_sumstats.columns.to_list()

['chr', 'pos', 'ref', 'alt', 'p_value']

In [20]:
df_sumstats['p_value'] = [1e-20 for i in range(len(df_sumstats))]

In [41]:
df_1kg = pd.read_csv('sumstats_head_1_2.tsv', sep='\t', skiprows=[1,50])

In [32]:
len(df_1kg)

279

In [48]:
df_1kg.tail()

,chromosome,base_pair_location,effect_allele,other_allele,beta,standard_error,effect_allele_frequency,p_value
322,1,931500,T,C,-0.003865,0.003412,0.884312,0.257285
323,1,931548,T,G,0.003497,0.002912,0.892707,0.229691
324,1,932457,A,G,0.003412,0.002921,0.889578,0.242733
325,1,933790,G,A,0.001937,0.002511,0.878090,0.440292
326,1,934121,A,G,0.003744,0.002942,0.874698,0.203229


In [43]:
df_1kg.columns.tolist()

['chromosome',
 'base_pair_location',
 'effect_allele',
 'other_allele',
 'beta',
 'standard_error',
 'effect_allele_frequency',
 'p_value']

In [46]:
df_1kg[df_1kg['chromosome']==22]

,chromosome,base_pair_location,effect_allele,other_allele,beta,standard_error,effect_allele_frequency,p_value


In [49]:
df_sumstats['alt'] = df_1kg['other_allele']
df_sumstats['ref'] = df_1kg['effect_allele']
df_sumstats['chr'] = df_1kg['chromosome']
df_sumstats['pos'] = df_1kg['base_pair_location']



In [51]:
gwas_snp_selection_fdr_bonf_method.main(coding_snp_list_path=path_exon, directory_1000_genomes=path_1KG_directory, track_list=gpt_track_list, df_summary_stats=df_sumstats, log_pvals=False)

ZeroDivisionError: float division by zero